In [ ]:

from tabulate import tabulate
import pandas as pd
   #
pd.set_option("display.width", 480)

print("--")


In [ ]:

from katana import remote

my_client = remote.Client()

print(my_client)


In [ ]:
NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"


l_cntr = 20


print("--")


In [17]:
#  CONNECT TO GRAPH

for l_graph in my_client.get_database(name=DB_NAME).graphs_in_database():
   if (l_graph.name == GRAPH_NAME):
      my_graph=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
         #
      break

print(my_graph)

<_Graph my_graph, FdwgFvpfC8GQdScQrdi5BYuGKY5Yvq5KR6aDRFWjFRTH, 2>


In [ ]:
######################################################

In [18]:

#  graph.project() is used to create a homogenous group, from a heterogeneous one,
#  necessary to support some analytics
#
#  Here we'll add stuff to the graph to make it heterogeneous.


l_query = """
   CREATE ( n: Restaurant { restaurant_code: 'PAP' } )               //  Notice single curly braces
   SET n.restaurant_name = 'Pappadeauxs Seafood Kitchen' 
   """
      #
display(l_result1 = my_graph.query(l_query))

l_query = """
   MATCH
      (n: Airport   ),
      (m: Restaurant)
   WHERE n.airport_code     = 'SJC'
   AND   m.restaurant_code  = 'PAP'
   CREATE (n) -[r: ATTRACTION { DRIVABLE: 'Yes' }]-> (m)
   """
      #
display(l_result2 = my_graph.query(l_query))


          0/? [?op/s]

          0/? [?op/s]

In [19]:

l_result1 = my_graph.query(
   """
   MATCH (n) 
   RETURN n
   """ )
print(tabulate(l_result1, headers='keys', tablefmt='psql'))

l_result2 = my_graph.query(
   """
   MATCH (n) - [r] -> (m)
   RETURN r
   """ )
print(tabulate(l_result2, headers='keys', tablefmt='psql'))


          0/? [?op/s]

+----+-----------------+----------------+-----------+------------------+------------------+----------+---------------------+-----------------------------+
|    |   n.internal_id | n.labels       | n.LABEL   | n.airport_code   | n.airport_name   | n.type   | n.restaurant_code   | n.restaurant_name           |
|----+-----------------+----------------+-----------+------------------+------------------+----------+---------------------+-----------------------------|
|  0 |               1 | ['Airport']    | Airport   | DEN              | Denver           | node     | nan                 | nan                         |
|  1 |               0 | ['Airport']    | Airport   | ORD              | Chicago O-Hare   | node     | nan                 | nan                         |
|  2 | 281474976710656 | ['Airport']    | Airport   | SJC              | San Jose         | node     | nan                 | nan                         |
|  3 | 281474976710657 | ['Restaurant'] | nan       | nan             

          0/? [?op/s]

+----+-----------------+----------------+-----------------+------------+--------------+--------------+----------+-----------------+------------------+----------+--------------+
|    |        r.end.id | r.end.labels   |   r.internal_id | r.label    |   r.DISTANCE |   r.NUM_HOPS | r.TYPE   |      r.start.id | r.start.labels   | r.type   | r.DRIVABLE   |
|----+-----------------+----------------+-----------------+------------+--------------+--------------+----------+-----------------+------------------+----------+--------------|
|  0 |               1 | ['Airport']    |               1 | FLIES_TO   |          886 |            1 | FLIES_TO |               0 | ['Airport']      | edge     | nan          |
|  1 | 562949953421312 | ['Airport']    |               3 | FLIES_TO   |           66 |            1 | FLIES_TO |               0 | ['Airport']      | edge     | nan          |
|  2 |               0 | ['Airport']    |               0 | FLIES_TO   |          886 |            1 | FLIES_TO |  

In [ ]:
######################################################

In [21]:

my_projection = my_graph.project(node_types=['Airport'], edge_types=['FLIES_TO'])

print("--")


--


In [22]:


l_result1 = my_projection.query(
   """
   MATCH (n) 
   RETURN n
   """ )
print(tabulate(l_result1, headers='keys', tablefmt='psql'))

l_result2 = my_projection.query(
   """
   MATCH (n) - [r] -> (m)
   RETURN r
   """ )
print(tabulate(l_result2, headers='keys', tablefmt='psql'))

#     OperationError: HaMw4a3njMMSvgQSnd2RL7r15TaavDnSvm1JePSRUjnh-5YoXGVdvYnT1L1qB5 Projected views are not supported for query operation yet. (QueryOperation.cpp:27): not implemented: not implemented



          0/? [?op/s]

OperationError: HaMw4a3njMMSvgQSnd2RL7r15TaavDnSvm1JePSRUjnh-5YoXGVdvYnT1L1qB5 Projected views are not supported for query operation yet. (QueryOperation.cpp:27): not implemented: not implemented

In [ ]:

graph_proj = graph.project(node_types=['Person'], edge_types=['KNOWS'])


#  Generate a unique prop name so we can run this over and over without edit ..
#
l_cntr += 1
   #
l_output_propname     = "sssp_dist_" + str(l_cntr).zfill(4)


prop_name = 'centrality'
analytics.betweenness_centrality(graph_proj, result_property_name=prop_name, num_sources=100)

result = graph.query(f"""
    MATCH (n)
    RETURN n, n.{prop_name}
    ORDER BY n.{prop_name} DESC
    """, contextualize=True)

result.view()


In [ ]:
l_result1 = my_graph.query(
   """
   MATCH (n) 
   RETURN n
   """ )
# l_result1.view()
print(tabulate(l_result1, headers='keys', tablefmt='psql'))
